In [1]:
using RigidBodyDynamics
using BenchmarkTools
using ForwardDiff

In [2]:
const joint = Joint("bla", QuaternionFloating{Float64}())

Joint "bla": Quaternion floating joint

In [3]:
# without autodiff:
@benchmark RigidBodyDynamics._local_coordinates!(jointType, ϕ, ϕ̇, q0, q, v) setup = (
    jointType = joint.jointType;
    q = Vector{Float64}(num_positions(joint));
    rand_configuration!(joint, q);
    q0 = Vector{Float64}(num_positions(joint));
    rand_configuration!(joint, q0);
    v = rand(num_velocities(joint));
    ϕ = Vector{Float64}(num_velocities(joint));
    ϕ̇ = Vector{Float64}(num_velocities(joint))
)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     250
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  480.00 bytes
  allocs estimate:  10
  minimum time:     300.00 ns (0.00% GC)
  median time:      337.00 ns (0.00% GC)
  mean time:        364.99 ns (4.91% GC)
  maximum time:     6.14 μs (93.36% GC)

In [4]:
const q̇ = Vector{Float64}(num_positions(joint))
const q_autodiff = Vector{ForwardDiff.Dual{1,Float64}}(num_positions(joint))
const q0_autodiff = Vector{ForwardDiff.Dual{1,Float64}}(num_positions(joint))
const ϕ_autodiff = Vector{ForwardDiff.Dual{1,Float64}}(num_velocities(joint))
function local_coordinates2!(jt::QuaternionFloating,
        ϕ::AbstractVector, ϕ̇::AbstractVector,
        q0::AbstractVector, q::AbstractVector, v::AbstractVector)
    RigidBodyDynamics._velocity_to_configuration_derivative!(jt, q̇, q, v)
    for i in eachindex(q)
        @inbounds q_autodiff = ForwardDiff.Dual(q[i], q̇[i])
        @inbounds q0_autodiff = ForwardDiff.Dual(q0[i], 0.)
    end
    RigidBodyDynamics._local_coordinates2!(jt, ϕ_autodiff, q0_autodiff, q_autodiff)
    for i in eachindex(ϕ_autodiff)
        @inbounds ϕ[i] = ForwardDiff.value(ϕ_autodiff[i])
        @inbounds ϕ̇[i] = ForwardDiff.partials(ϕ_autodiff[i])[1]
    end
end

local_coordinates2! (generic function with 1 method)

In [5]:
# with autodiff:
@benchmark local_coordinates2!(jointType, ϕ, ϕ̇, q0, q, v) setup = (
    jointType = joint.jointType;
    q = Vector{Float64}(num_positions(joint));
    rand_configuration!(joint, q);
    q0 = Vector{Float64}(num_positions(joint));
    rand_configuration!(joint, q0);
    v = rand(num_velocities(joint));
    ϕ = Vector{Float64}(num_velocities(joint));
    ϕ̇ = Vector{Float64}(num_velocities(joint))
)

BenchmarkTools.Trial: 
  samples:          10000
  evals/sample:     192
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  528.00 bytes
  allocs estimate:  11
  minimum time:     532.00 ns (0.00% GC)
  median time:      547.00 ns (0.00% GC)
  mean time:        583.14 ns (3.74% GC)
  maximum time:     8.76 μs (93.35% GC)

In [6]:
using ProfileView

In [7]:
const q = Vector{Float64}(num_positions(joint));
rand_configuration!(joint, q);
const q0 = Vector{Float64}(num_positions(joint));
rand_configuration!(joint, q0);
const v = rand(num_velocities(joint));
const ϕ = Vector{Float64}(num_velocities(joint));
const ϕ̇ = Vector{Float64}(num_velocities(joint));

In [10]:
f1 = CartesianFrame3D(); f2 = CartesianFrame3D(); t1 = rand(Transform3D{Float64}, f1, f2); t2 = rand(Transform3D{Float64}, f1, f2)
@code_warntype *(t1, t2)

Variables:
  #self#::Base.#*
  t1::RigidBodyDynamics.Transform3D{Float64}
  t2::RigidBodyDynamics.Transform3D{Float64}
  rot::Quaternions.Quaternion{Float64}
  trans::StaticArrays.SVector{3,Float64}
  qvec::StaticArrays.SVector{3,Float64}
  #203@_7::StaticArrays.##203#204{Base.#*,Float64}
  #temp#@_8::Float64
  #203@_9::StaticArrays.##203#204{Base.#*,Float64}
  #temp#@_10::Float64
  #203@_11::StaticArrays.##203#204{Base.#*,Float64}

Body:
  begin 
      $(Expr(:invoke, LambdaInfo for framecheck(::RigidBodyDynamics.CartesianFrame3D, ::RigidBodyDynamics.CartesianFrame3D), :(RigidBodyDynamics.framecheck), :((Core.getfield)(t1,:from)::RigidBodyDynamics.CartesianFrame3D), :((Core.getfield)(t2,:to)::RigidBodyDynamics.CartesianFrame3D))) # line 103:
      rot::Quaternions.Quaternion{Float64} = $(Expr(:invoke, LambdaInfo for *(::Quaternions.Quaternion{Float64}, ::Quaternions.Quaternion{Float64}), :(RigidBodyDynamics.*), :((Core.getfield)(t1,:rot)::Quaternions.Quaternion{Float64}), :((Core.getf

In [8]:
@code_warntype RigidBodyDynamics._local_coordinates!(joint.jointType, ϕ, ϕ̇, q0, q, v)

Variables:
  #self#::RigidBodyDynamics.#_local_coordinates!
  jt::RigidBodyDynamics.QuaternionFloating{Float64}
  ϕ::Array{Float64,1}
  ϕ̇::Array{Float64,1}
  q0::Array{Float64,1}
  q::Array{Float64,1}
  v::Array{Float64,1}
  frameBefore::RigidBodyDynamics.CartesianFrame3D
  frame0::RigidBodyDynamics.CartesianFrame3D
  frameAfter::RigidBodyDynamics.CartesianFrame3D
  t0::RigidBodyDynamics.Transform3D{Float64}
  t::RigidBodyDynamics.Transform3D{Float64}
  relative_transform::RigidBodyDynamics.Transform3D{Float64}
  rot::Quaternions.Quaternion{Float64}
  trans::StaticArrays.SVector{3,Float64}
  Θ_over_2::Float64
  Θ::Float64
  sΘ_over_2::Float64
  cΘ_over_2::Float64
  axis::StaticArrays.SVector{3,Float64}
  ϕrot::StaticArrays.SVector{3,Float64}
  α::Float64
  Θ_squared::Float64
  p::StaticArrays.SVector{3,Float64}
  ϕtrans::StaticArrays.SVector{3,Float64}
  ω@_26::StaticArrays.SVector{3,Float64}
  ν::StaticArrays.SVector{3,Float64}
  β::Float64
  A::Float64
  B::Float64
  #temp#@_31::Int

In [ ]:
function test()
    for i = 1 : 1000000
        local_coordinates2!(joint.jointType, ϕ, ϕ̇, q0, q, v)
    end
end

In [ ]:
test()
Profile.clear()
@profile test()
ProfileView.view()

In [ ]:
@benchmark b * a setup = (a = rand(); b = ForwardDiff.Dual(rand(), rand()))

In [ ]:
@benchmark inv(t1) * t2 setup = (f1 = CartesianFrame3D(); f2 = CartesianFrame3D(); t1 = rand(Transform3D{Float64}, f1, f2); t2 = rand(Transform3D{Float64}, f1, f2))